## Imports

In [ ]:
import json
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
from numpy import linalg as LA

import pandas as pd

import networkx as nx
import community

from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.metrics import adjusted_rand_score

## Data preparation
### Loading of the data

In [ ]:
# Download names to make it more friendly
json_file_path_1 = '../data/all_coins_by_mc_1.json'
json_file_path_2 = '../data/all_coins_by_mc_2.json'

# Open the JSON file and load its contents
with open(json_file_path_1, 'r') as file:
    data = json.load(file)

with open(json_file_path_2, 'r') as file:
    data = data + json.load(file)
# Now, 'data' contains the contents of the JSON file as a Python object (dictionary, list, etc.)
names_list = [stock["name"] for stock in data]

### Load price data and preprocessing

In [ ]:
data = pd.read_csv("../data/processed/prices.csv", parse_dates=['date'])
data.rename(columns={data.columns[0]: 'date'}, inplace=True)
data.set_index("date", inplace=True)
data.columns = names_list
data = data.loc['2021-01-01':'2022-12-31'] # Filter
data = data.dropna(axis=1, how='all') # Drop all columns that contains only null value
data = data.dropna(axis=1, thresh=0.95*len(data.index)) # Drop columns that contains more than 25% of null value
data = data.ffill() # ffill null values
data.dropna(inplace=True)

### Compute and normalize Hourly log-return

In [ ]:
hourly_returns_pct = data.pct_change()
hourly_returns_pct = hourly_returns_pct.iloc[1:]
hourly_returns_pct.to_csv("../data/processed/hourly_return.csv")
hourly_log_returns = np.log(hourly_returns_pct + 1)

In [ ]:
scaler = StandardScaler()
hourly_log_returns = pd.DataFrame(scaler.fit_transform(hourly_log_returns), columns=hourly_log_returns.columns, index=hourly_log_returns.index)
hourly_log_returns.to_csv("../data/processed/normalized_log_ret.csv")

## Eigenvalues of correlation matrix analysis

In [ ]:
plt.subplots(figsize = (12,6))
sns.histplot(hourly_log_returns.values.flatten(), stat='density', bins=2000, alpha=0.5, color="blue", edgecolor='None')
sns.histplot(np.random.normal(0,1,1000000), stat='density', alpha=0.5, bins = 30, color="orange", edgecolor='None')

plt.yscale('log')
plt.xlim(-25, 25)
plt.show()

In [ ]:
# Function to permute each column independently
def permute_columns(df):
    permuted_df = df.apply(np.random.permutation, axis=0)
    return permuted_df

In [ ]:
def P0(lambdas,q, sigma):
    lambda_plus = sigma**2*(1+np.sqrt(q))**2
    lambda_minus = sigma**2*(1-np.sqrt(q))**2
    vals = 1/(q*2*np.pi*sigma**2*lambdas)*np.sqrt((lambda_plus-lambdas)*(lambdas-lambda_minus))
    return vals

##### Random

In [ ]:
N=1000
T=7000
R=np.random.normal(0,1,N*T).reshape((N,T))
R
R.shape

In [ ]:
C=np.corrcoef(R)
C.shape

In [ ]:
lambdas_e, V_e = LA.eig(C)

In [ ]:
q=N/T
lambdas=np.linspace((1.-np.sqrt(q))**2,(1.+np.sqrt(q))**2,200)
P0s=P0(lambdas, q, 1)

In [ ]:
sns.distplot(lambdas_e[1:],kde=False, norm_hist=True,bins=50)  # no Kernel Density Estimation
plt.plot(lambdas,P0s)

In [ ]:
lambdas_e_list_720 = []
for i in range(1):#range(100, 11000, 100):
    permuted_df = hourly_log_returns[i:720+i]
    (T_720,N_720) = permuted_df.shape
    C = permuted_df.corr().to_numpy()
    lambdas_e, V_e = LA.eig(C)
    lambdas_e_list_720.append(lambdas_e)
lambdas_e_list_720 = np.array(lambdas_e_list_720).flatten()

In [ ]:
lambdas_e_list_perm = []
for i in range(100):
    permuted_df = permute_columns(hourly_log_returns)
    (T_perm,N_perm) = permuted_df.shape
    C = permuted_df.corr().to_numpy()
    lambdas_e, V_e = LA.eig(C)
    lambdas_e_list_perm.append(lambdas_e)
lambdas_e_list_perm = np.array(lambdas_e_list_perm).flatten()

In [ ]:
(T_all,N_all) = hourly_log_returns.shape
C = hourly_log_returns.corr().to_numpy()
lambdas_e_all, V_e = LA.eig(C)

In [ ]:
q_all=N_all/T_all
sigma = 1
lambdas_all=np.linspace(sigma**2*(1.-np.sqrt(q_all))**2,sigma**2*(1.+np.sqrt(q_all))**2,200)
P0s_all=P0(lambdas_all, q_all, sigma=sigma)

q_perm=N_perm/T_perm
sigma = 1
lambdas_perm=np.linspace(sigma**2*(1.-np.sqrt(q_perm))**2,sigma**2*(1.+np.sqrt(q_perm))**2,200)
P0s_perm=P0(lambdas_perm, q_perm, sigma=sigma)

q_720=N_720/T_720
sigma = 1
lambdas_720=np.linspace(sigma**2*(1.-np.sqrt(q_720))**2,sigma**2*(1.+np.sqrt(q_720))**2,200)
P0s_720=P0(lambdas_720,q_720, sigma=sigma)

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(14,4), sharey=True)

sns.histplot(lambdas_e_all, ax = axs[0], stat='density', bins=1000, color='#EEA47F', label='Empirical distribution')  # no Kernel Density Estimation
axs[0].plot(lambdas_all, P0s_all, color='#00539C', label='Marcenko-Pastur distribution')
axs[0].set_xlim(0,6)
axs[0].set_title("Full correlation matrix")

plt.legend()



sns.histplot(lambdas_e_list_perm, ax = axs[1], stat='density', bins=50, color='#EEA47F', label='Empirical distribution')  # no Kernel Density Estimation
axs[1].plot(lambdas_perm, P0s_perm, color='#00539C', label='Marcenko-Pastur distribution')
axs[1].set_title("Full correlation matrix (with permutation)")




sns.histplot(lambdas_e_list_720, ax = axs[2], stat='density', bins=400, color='#EEA47F', label='Empirical distribution')  # no Kernel Density Estimation
axs[2].plot(lambdas_720, P0s_720, color='#00539C', label='Marchenko–Pastur distribution')
axs[2].set_xlim(0, 6)
axs[2].set_title("T = 720 (1 month)")


fig.suptitle('Density of the eigenvalues', fontsize=16, y = 1.04)


plt.legend()
plt.show()

## Network creation methods

### Filtering based on RMT

In [ ]:
def compute_C_minus_C0(lambdas, v, lambda_plus, removeMarketMode=False):
    N=len(lambdas)
    C_clean=np.zeros((N, N))

    order = np.argsort(lambdas)
    lambdas,v = lambdas[order],v[:,order]

    v_m=np.matrix(v)

    # note that the eivenvalues are sorted
    for i in range(1*removeMarketMode,N):
        if lambdas[i]>lambda_plus:
            C_clean=C_clean+lambdas[i] * np.dot(v_m[:,i],v_m[:,i].T)
    return C_clean

### From C (as in class)

In [ ]:
def baseline_network(R, mst=False):   # R is a matrix of return
    
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2
    C=R.corr()
    lambdas, v = LA.eigh(C)
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    C_s = np.abs(C_s)

    return nx.from_numpy_array(C_s)


### Minimum Spanning Tree

In [ ]:
def mst_network(R):
    N=R.shape[1]
    T=R.shape[0]

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2
    C=R.corr()
    lambdas, v = LA.eigh(C)
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)
    
    # Compute distance matrix
    D = np.sqrt(2*(1-C_s))
    # Compute MST
    G = nx.from_numpy_array(D)
    return nx.minimum_spanning_tree(G)

### Threshold network
- Compute C and D
- Create G from D
- Remove all edge with weight smaller than *threshold*
- Louvain clustering

In [ ]:
def threshold_network(R, threshold=0.05):
    N = R.shape[1]
    T = R.shape[0]
    
    # Compute lambda_plus and correlation matrix
    q = N * 1. / T
    lambda_plus = (1. + np.sqrt(q)) ** 2
    C = R.corr()
    lambdas, v = LA.eigh(C)
    C_s=compute_C_minus_C0(lambdas,v,lambda_plus)

    # Compute distance matrix
    D = np.sqrt(2*(1-C_s))    
    
    # Compute graph
    G = nx.from_numpy_array(D)
    
    G_copy = G.copy()

    # Get the edge weights as a dictionary
    edge_weights = nx.get_edge_attributes(G_copy, 'weight')

    # Calculate the threshold for the top 5% of edges
    threshold = sorted(edge_weights.values())[int(threshold * len(edge_weights))]
    # Identify edges to remove based on the threshold
    edges_to_remove = [(u, v) for (u, v, w) in G_copy.edges(data='weight') if w > threshold]
    
    G.remove_edges_from(edges_to_remove)
    print(nx.number_connected_components(G))
    print(G)
    return G

### Planar Maximally Filtered Graph

In [ ]:
def sort_graph_edges(G):
    sorted_edges = []
    for source, dest, data in sorted(G.edges(data=True), key=lambda x: x[2]['weight']):
        sorted_edges.append({'source': source,
                             'dest': dest,
                             'weight': data['weight']})
        
    return sorted_edges

def compute_PMFG(sorted_edges, nb_nodes):
    PMFG = nx.Graph()
    for edge in sorted_edges:
        PMFG.add_edge(edge['source'], edge['dest'])
        if not nx.is_planar(PMFG):
            PMFG.remove_edge(edge['source'], edge['dest'])
            
        if len(PMFG.edges()) == 3*(nb_nodes-2):
            print('test')
            break
    
    return PMFG


def pmfg_network(R):
    N = R.shape[1]
    T = R.shape[0]
    
    # Compute lambda_plus and correlation matrix
    q = N * 1. / T
    lambda_plus = (1. + np.sqrt(q)) ** 2
    C = R.corr()
    lambdas, v = LA.eigh(C)
    C = compute_C_minus_C0(lambdas, v, lambda_plus)

    # Compute distance matrix
    D = np.sqrt(2*(1-C))    
    
    # Compute graph
    G = nx.from_numpy_array(D)
    
    sorted_edges = sort_graph_edges(G)
    
    return compute_PMFG(sorted_edges, N)

## Louvain clustering

In [ ]:
def louvain_clustering(G, resolution):
    return  community.community_louvain.best_partition(G, resolution=resolution)

## Rolling window

In [ ]:
df = pd.read_csv("../data/processed/normalized_log_ret.csv")

In [ ]:
def rename_asset(R, partition):
    dict_cluster = {}
    all_names = list(R.columns)
    for i, name in enumerate(all_names):
        dict_cluster[name]=partition[i]
        
    return dict_cluster

def clustering(R: pd.DataFrame, period: int=720, interval: int = 24, method:str='baseline') -> dict:
    """Compute the clusters in a rolling window manner for the return matrix R.

    Args:
        R (pd.Dataframe): Normalized log-return matrix
        period (int, optional): time period_. Defaults to 30 days (720 hours).
        interval (int, optional): time interval. Defaults to 1 day (24 hours).
        method (str, optional): Clustering method. Defaults to 'baseline'. Can be 'baseline', 'mst' 'threshold', 'pmf'.
    """
    cluster_dict = {}
    resolution = 1
    for t0 in tqdm(range(0, len(R.index)-period, interval)):
        R_tmp = R.iloc[t0:t0+period]
        if method == 'baseline':
            G = baseline_network(R_tmp)
            resolution = 1.02
        elif method == 'mst':
            G = mst_network(R_tmp)
            resolution = TODO
        elif method == 'threshold':
            G = threshold_network(R_tmp, threshold=0.3)
            resolution = 1
        elif method == 'pmf':
            G = pmfg_network(R_tmp)
            resolution = TODO
        else:
            raise ValueError("Method not recognized")
            
        
        cluster_dict[(t0, t0+period)] = rename_asset(R_tmp, louvain_clustering(G, resolution))
            
    return cluster_dict

In [73]:
clusters = clustering(df.set_index('date'), period=720, interval=24, method='threshold')
clusters

100%|███████████████████████████████████████████| 10/10 [00:07<00:00,  1.29it/s]


{(0, 720): {'Bitcoin': 0,
  'Ethereum': 0,
  'Tether': 1,
  'BNB': 0,
  'XRP': 0,
  'Solana': 0,
  'USDC': 2,
  'Lido Staked Ether': 0,
  'Cardano': 0,
  'Dogecoin': 0,
  'TRON': 0,
  'Chainlink': 0,
  'Avalanche': 0,
  'Polygon': 0,
  'Polkadot': 0,
  'Wrapped Bitcoin': 2,
  'Dai': 2,
  'Litecoin': 0,
  'Shiba Inu': 1,
  'Uniswap': 0,
  'Bitcoin Cash': 0,
  'LEO Token': 2,
  'OKB': 0,
  'Stellar': 0,
  'Monero': 0,
  'TrueUSD': 2,
  'Cosmos Hub': 0,
  'Ethereum Classic': 0,
  'Cronos': 0,
  'Filecoin': 0,
  'Lido DAO': 2,
  'THORChain': 0,
  'Hedera': 0,
  'NEAR Protocol': 2,
  'BUSD': 1,
  'VeChain': 0,
  'Injective': 2,
  'Aave': 0,
  'Quant': 0,
  'Maker': 3,
  'The Graph': 1,
  'Render': 2,
  'MultiversX': 0,
  'Synthetix Network': 0,
  'Algorand': 0,
  'Stacks': 2,
  'Theta Network': 0,
  'Flow': 1,
  'Bitcoin SV': 2,
  'Fantom': 1,
  'The Sandbox': 2,
  'Axie Infinity': 2,
  'IOTA': 1,
  'Decentraland': 1,
  'WEMIX': 0,
  'Terra Luna Classic': 0,
  'NEO': 0,
  'EOS': 0,
  'KuCoi

In [ ]:
import pickle 

# Save the clusters
name = "thresh_720_24_0dot3_1.pkl"
with open("../data/processed/"+name, 'wb') as f:
    pickle.dump(clusters, f)

In [ ]:
# Inspect first n cluster

n = 10
for i in range(n):
    dict_clusters = clusters[list(clusters.keys())[i]]
    grouped_cryptos = {}

    for crypto, number in dict_clusters.items():
        if number not in grouped_cryptos:
            grouped_cryptos[number] = []
        grouped_cryptos[number].append(crypto)

    # Convert the dictionary values to a list for easier usage
    result_lists = list(grouped_cryptos.values())

    # Print or use the result_lists as needed
    print("Clustering {} \n".format(i + 1))
    for c in range(len(result_lists)):
        print(result_lists[c])
        print()

## Stability

In [ ]:
def compute_rand_score(clusters):
    nb_clusters = len(clusters.keys())
    rand_score = []
    for i in range(nb_clusters - 1):
        score = adjusted_rand_score(list(clusters[list(clusters.keys())[i]].values()), list(clusters[list(clusters.keys())[i+1]].values()))
        rand_score.append(score)
    return rand_score

In [ ]:
def compute_nb_clusters(clusters):
    nb_clusters = len(clusters.keys())
    list_nb_clusters = []
    for i in range(nb_clusters - 1):
        nb = max(clusters[list(clusters.keys())[i]].values())
        list_nb_clusters.append(nb)
    return list_nb_clusters

In [ ]:
compute_nb_clusters(clusters)

In [ ]:
# Nb of clusters
plt.boxplot(compute_nb_clusters(clusters))
plt.show()

In [ ]:
# Stability of the clusters
plt.boxplot(compute_rand_score(clusters))
plt.show()

## Markowitz

In [ ]:
def find_representatives(clusters):
    nb_clustering = len(clusters.keys())
    representatives = []
    for i in range(nb_clustering):
        
        current_representatives = []
        current_cluster = clusters[list(clusters.keys())[i]]

        # Loop through each community
        for group in range(max(current_cluster.values())):
            subset = [cluster[0] for cluster in current_cluster.items() if cluster[1] == group]
            # Subset the data for the current group
            subset_data = df[subset]

            imputer = SimpleImputer(strategy='mean')
            subset_data_imputed = pd.DataFrame(imputer.fit_transform(subset_data), columns=subset_data.columns, index=subset_data.index)

            # Standardize/Normalize the data
            scaler = StandardScaler()
            subset_data_standardized = scaler.fit_transform(subset_data_imputed)

            # Apply PCA
            pca = PCA()
            principal_components = pca.fit_transform(subset_data_standardized)

            # Identify the leading coin (biggest contributor in the first principal component)
            leading_coin_index = np.argmax(np.abs(pca.components_[0]))
            leading_coin = subset_data.columns[leading_coin_index]
            current_representatives.append(leading_coin)
            # Print
        representatives.append(current_representatives)
    return representatives

In [ ]:
representatives = find_representatives(clusters)
representatives

In [ ]:
from collections import Counter
Counter([a for r in representatives for a in r]).most_common(100)

In [ ]:
len(representatives)

In [ ]:
def out_sample_risks(clusters):

    nb_clustering = len(clusters.keys())
    vol = []
    vol_abs = []
    for i in range(nb_clustering):
        length_window = list(clusters.keys())[i][1] - list(clusters.keys())[i][0]

        corr_mat_insample = df.loc[list(clusters.keys())[i][0]:list(clusters.keys())[i][1],representatives[i]].corr()
        corr_mat_outsample = df.loc[list(clusters.keys())[i][1]:list(clusters.keys())[i][1] + length_window, representatives[i]].corr()

        inv_corr_insample = LA.inv(corr_mat_insample.to_numpy())
        w_opt = inv_corr_insample @ np.ones(len(inv_corr_insample)) / (np.ones(len(inv_corr_insample)) @ inv_corr_insample @ np.ones(len(inv_corr_insample)))

        realized_vol = w_opt @ corr_mat_insample @ w_opt.T
        out_sample_vol = w_opt @ corr_mat_outsample @ w_opt.T
        vol.append((realized_vol, out_sample_vol))
        vol_abs.append(abs(realized_vol-out_sample_vol))
    return vol, vol_abs

In [ ]:
vol, vol_abs = out_sample_risks(clusters)
vol_abs

In [ ]:
# RMT out_sample risks

nb_clustering = len(clusters.keys())
vol_rmt = []
vol_abs_rmt = []
for i in range(nb_clustering):
    length_window = list(clusters.keys())[i][1] - list(clusters.keys())[i][0]

    N=df.shape[1]
    T=length_window

    q=N*1./T
    lambda_plus=(1.+np.sqrt(q))**2
    
    corr_mat_insample = df.loc[list(clusters.keys())[i][0]:list(clusters.keys())[i][1],:].drop('date', axis=1).corr()
    lambdas, v = LA.eigh(corr_mat_insample)
    corr_mat_insample=np.array(compute_C_minus_C0(lambdas,v,lambda_plus))
    
    corr_mat_outsample = df.loc[list(clusters.keys())[i][1]:list(clusters.keys())[i][1] + length_window, :].drop('date', axis=1).corr()
    lambdas, v = LA.eigh(corr_mat_outsample)
    corr_mat_outsample=np.array(compute_C_minus_C0(lambdas,v,lambda_plus))
        
    inv_corr_insample = LA.inv(corr_mat_insample)
    w_opt = inv_corr_insample @ np.ones(len(inv_corr_insample)) / (np.ones(len(inv_corr_insample)) @ inv_corr_insample @ np.ones(len(inv_corr_insample)))

    realized_vol = w_opt @ corr_mat_insample @ w_opt.T
    out_sample_vol = w_opt @ corr_mat_outsample @ w_opt.T
    vol_rmt.append((realized_vol, out_sample_vol))
    vol_abs_rmt.append(abs(realized_vol-out_sample_vol))
vol_abs_rmt